In [1]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
# Load the data into a DataFrame
data = pd.read_csv(filepath_or_buffer='final_data_only_tpm.csv')

# Split the data into features (X) and target variable (y)
X = data.drop('cancer_type', axis=1)
y = data['cancer_type']


In [2]:

# Split the data into training and validation sets
train_X, test_y = train_test_split(X, y, test_size=0.2, random_state=42)

data.head()

,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,...,AC092910.4,AC073611.1,AC136977.1,AC078856.1,AC008763.4,AL592295.6,AC006486.3,AL391628.1,AP006621.6,cancer_type
0,19.7011,0.1125,46.1397,1.8913,0.2670,18.6466,4.3945,24.1591,6.7242,9.4371,...,0.0,0.0641,0.0,1.0888,0.0,3.1977,0.0,0.0104,0.3953,kidney chromophobe
1,31.6275,0.0000,45.5189,2.8743,0.4744,4.2348,1.4145,60.8153,15.4309,7.5267,...,0.0,0.0781,0.0,0.0000,0.0,15.4262,0.0,0.0042,0.4094,kidney chromophobe
2,35.9928,0.5207,108.0270,4.8134,0.8010,4.5446,3.2850,33.1630,13.1085,12.1888,...,0.0,0.1049,0.0,0.0000,0.0,13.2312,0.0,0.1154,0.3050,kidney chromophobe
3,18.2605,0.0921,27.5398,2.3258,0.3473,4.5023,1.5078,47.2006,10.6721,5.0657,...,0.0,0.1193,0.0,0.0000,0.0,6.4516,0.0,0.0141,0.1416,kidney chromophobe
4,68.8052,3.4134,88.0619,6.2415,1.2251,9.5310,30.6990,66.3841,12.9370,20.1289,...,0.0,0.1141,0.0,0.0000,0.0,21.8857,0.0,0.0394,0.3547,kidney chromophobe


In [ ]:
#convert to tensors
import torch


X_train_tens = torch.from_(X_np)
X_train_tens = X_train_tens.type(torch.FloatTensor)
y_train_tens = torch.from_numpy(y_np)
y_train_tens = y_train_tens.type(torch.LongTensor)

device = torch.device('cpu' if torch.cuda.is_available() else 'cpu')

batch_size = 10
input_size = 154
hiden_size = 462
num_classes = 4
learning_rate = 0.001
num_epochs = 100

input_train = autograd.Variable(X_train_tens)
target_train = autograd.Variable(y_train_tens)

#define FNN

class Net(nn.Module):
    def __init__(self,input_size,hiden_size,num_classes):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hiden_size)
        self.fc2 =nn.Linear(hiden_size, num_classes)
    
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

model = Net(input_size=input_size, hiden_size=hiden_size, num_classes=num_classes).to(device)
opt = torch.optim.Adam(params=model.parameters(), lr=learning_rate)
loss_func = torch.nn.CrossEntropyLoss()

#run training stage

correct = 0
total = 0
counter = 0
for epoch in range (num_epochs):
    out = model(input_train).to(device)
    _, pred = out.max(1)
    total += target_train.size(0)
    correct += (pred == target_train).sum().item()
    print(input_train)
    print(pred)
    loss = loss_func(out,target_train)
    counter +=1
    print('loss train', "Epoch N", counter,loss.data[0])
    model.zero_grad()
    loss.backward()
    opt.step()
print('Accuracy of the network on train dataset: {} %'.format(100 * correct / total))